# Parameters

In [ ]:
features_save_path = 'extracted-features.h5'
model_save_path = 'tomato_model.cpickle'


# Import saved features

In [ ]:
import h5py

db = h5py.File(output)
print(list(db.keys()))

# Train logistical regression classifier

In [ ]:
# import packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pickle
import h5py

jobs = -1

# open the HDF5 database for reading then determine the index of the training and
# testing split, provided that this data was already shuffled *prior* to writing it to disk
db = h5py.File(features_save_path, "r")

# reserve 75% of dataset for training (index i is the index at which training data end)
# no need to shuffle because it was done earlier (before putting to VGG)
i = int(labels.shape[0] * 0.75)

# define the set of parameters that we want to tune then start a grid
# search where we evaluate our model for each value of C
print("[INFO] tuning hyperparameters...")
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
model = GridSearchCV(LogisticRegression(), params, cv = 3, n_jobs = jobs)

# db["features"][:i] Data before index i is training data.
model.fit(db["features"][:i], db["labels"][:i])
print("[INFO] best hyperparameters: {}".format(model.best_params_))

# evaluate the model
print("[INFO] evaluating...")
# db["features"][i:] Data after index i is testing data.
preds = model.predict(db["features"][i:])
print(classification_report(db["labels"][i:], preds, target_names = db["label_names"]))

# serialize the model to disk
print("[INFO] saving model...")
f = open(model, "wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()

# close the dataset
db.close()